# Modelo de Black-Litterman

El modelo propuesto va a permitir incluir información cualitativa en el CAPM. Se asume que la matriz de covarianza es dada y conocida, y nos centramos en las primas de riesgo. Lógicamente desde la perspectiva del CAPM el modelo es inutil pues apuesta claramente por una estrategia pasiva. Una estrategia de inversión activa precisa de previsiones sobre la evolución de alguno de los títulos. 

Partimos la función de utilidad: 

$$U=w^T\Pi -\frac{\lambda}{2}w^T\Sigma w \tag{1}$$

- $U$ es la función de utilidad que deseamos optimizar de acuerdo al CAPM
- $w$ vector de ponderaciones
- $\Pi$ vector de premio de mercado o exceso de rentabilidad 
- $\lambda$ aversión al riesgo
- $\Sigma$ matriz de covarianza

La función objetivo que utilizada para la identificación de las carteras eficientes:

$$\underset{w}{max}w^T \Pi-\frac{\lambda}{2} w^T \Sigma w \tag{2}$$

Dado que $U$ es convexa aplicamos la primera condición de optimización:

$$\frac{dU}{dw}=\Pi -\lambda\Sigma w=0 \tag{3}$$

La primas por riesgo en equilibrio vienen dado por:

$$\Pi=\lambda\Sigma w \tag{4}$$

Con el objeto de estimar la cartera de mercado necesito el valor de la aversión al riesgo del mercado $\lambda$. Multiplico ambos lados de la ecuación por $w^T$:

$$(r_M-r_f)=\lambda \sigma^2 \tag{5}$$

Muchos autores determinan el valor de $\lambda$. Black y Litterman (1992) utilizan un valor próximo a 0.5. Podemos definir la formula 5 en función del ratio de Sharpe (SR):

$$\lambda=\frac{SR}{\sigma_M} \tag{5}$$

De cualquier modo podemos estimar las ponderaciones de la cartera M:

$$w_M=\frac{1}{\lambda}\Sigma^{-1}\Pi \tag{6}$$

Necesitamos estimar la varianza de la cartera $\Sigma_\Pi$. Black-Litterman proponen una simplificación, la matriz de covarianzas es proporcional a la matriz de covarianzas histórica $\Sigma$. Por tanto, definimos una parámetro $\tau$ que es una constante y $\Sigma_\Pi=\tau\Sigma$.

El modelo de Black-Litterman nos permite integrar, a través de la estadística bayesiana, dos funciones de distribución, una vinculada la información histórica y otra relacionada con la opiniones subjetivas. Así, la ley de Bayes propone:

$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}\tag{7}$$

donde $P(A|B)$ es la probabilidad de A condicionada a que ocurra B, $P(B|A)$ es la probabiliad de B condicionada a que ocurra A, $P(A)$ y $P(B)$ son las probabilidades no condicionadas. En términos generales $P(A)$ se denomina distribución a priori y $P(B|A)$ es la probabilidad condicional de los datos observados a priori.

El primer componente, la distribución a priori se define de la siguiente manera:

$$ P(A)\sim N(r_e,\tau \Sigma), r_A\sim N(r_e, \Sigma)\tag{8} $$

Estamos indicando que la probabilidad a priori se distribuye como una normal cuya esperanza son las rentabilidades esperadas en equilibrio y su varianza es proporcional a la matriz de covarianzas histórica. Representa nuestra estimación de la media, que se expresa como una distribución de la media real desconocida sobre nuestra estimación.

Podemos redefinir la fórmula 8:

$$ r_A\sim N(r_e,(1+\tau) \Sigma) \tag{9} $$

La rentabilidad estimada es la resultante de crear una cartera que invierte $1/(1+\tau)$ en la cartera de mercado y $\tau/(1+\tau)$ en el activo libre de riesgo:

$$\begin{aligned}
\Pi&=\lambda\Sigma w \\
w&=(\lambda\Sigma)^{-1} \Pi \\
\hat{w}&=((1+\tau)\lambda\Sigma)^{-1} \Pi \\
\hat{w}&=(1/1+\tau)(\lambda\Sigma)^{-1} \Pi \\
\hat{w}&=(1/1+\tau)w
\end{aligned}$$

El segundo componente, la probabilidad condicional, son las opiniones y se expresarán como:

$$P(B|A)\sim N(P^TQ, P^T\Omega P) \tag{16}$$

La distribución posterior, es la distribución de las rentabilidades considerando las rentabilidad de equilibrio y las opiniones. Se estimará como el producto de las dos distribuciones previas y se podrá expresar así:
$$\begin{aligned}
P(A|B) &\sim N(\mathbb{E}[r], var(r)) \tag{17}\\
(\mathbb{E}[r]=
\end{aligned}$$

Figura 1: Comparación entre la cartera de mercado propuesta por el CAPM y la cartera oculta de Black-Litterman.

![CML](https://raw.githubusercontent.com/alfonso-santos/Python-aplicado-gestion-carteras/fernando/Tema_2/imgs/c10f001.jpg)

Fuente:Brandimarte(2018)



Definimos $w_{Mj}$ como la ponderación del activo $j$ en la cartera de mercado $M$. Por tanto la rentabilidad de la cartera de mercado viene definida por:

$$\tilde{r}_M=\sum_{j=1}^n w_{Mj}\tilde{r}_j \tag{3}$$

De acuerdo al CAPM:

$$\mu_j-r_f=\beta_j(\mu_M-r_f) \tag{4}$$

donde

$$\mu_M=\sum_{j=1}^n w_{Mj}\mu_j \tag{5}$$

El premio por el riesgo será:

$$\pi_{ej}=\mu_j-r_f \tag{6}$$

el subíndice $e$ se utiliza para enfatizar que se trata de un mercado en equilibrio. Si desarrollamos la ecuación anterior:

$$\begin{gather}
\pi_{ej}=\frac{Cov(\tilde{r}_i,\tilde{r}_M)}{\sigma_M^2}(\mu_M-r_f) \tag{7} \\
=\frac{1}{\sigma_M^2}Cov\left(\tilde{r}_i,\sum_{j=1}^{n} w_{Mj}\tilde{r}_j \right)(\mu_M-r_f) \\ 
=\frac{\mu_M-r_f}{\sigma_M^2} \sum_{j=1}^{n} w_{Mj} Cov(\tilde{r}_i,\tilde{r}_j ) 
\end{gather}$$





Podemos expresar matricialmente la ecuación 7:

$$\pi_e=\delta\Sigma w_M \tag{8}$$

y definimos el precio de mercado del riesgo

$$\delta=\frac{\mu_M-r_f}{\sigma_M^2} \tag{9}$$

Si aplicamos la estadística Bayesiana la prima por riesgo $pi_e$ es considerada como un vector de variables aleatorias $\tilde{\theta}$ condicionada por la información disponible $\mathcal{M}_e$, por tanto:

$$\pi_e=E\left[\tilde{\theta}\left. \right|\mathcal{M}_e\right] \tag{10}$$

La notación $\pi_S$ representa la prima por riesgo subjetiva y $\pi_{BL}$ es la prima por riesgo propuesta por Black-Literrman. Suponemos que la rentabilidad y los parámetros se distribuyen como normales.

Tras un profundo análisis de la realidad empresarial el análisis propone que el activo 2 batirá la rentabilidad del activo 5 en un 2%, es decir:
$$\pi_{s2}-\pi_{s5}=0.02 \tag{11}$$

La propuesta subjetiva sobre la prima por riesgo puede expresarse como una relación lineal, desde una perspectiva matricial sería:

$$P\pi_{s}=Q \tag{12}$$

donde $P\in \mathbb{R}^{k\times n}$, $Q\in \mathbb{R}^k$, y $k$ es el número de opiniones subjetivas. Por tanto, $P$ representa las opiniones y $Q$ las primas resultantes tras incorporar las opiniones.




Lógicamente las opiniones subjetivas son aleatorias lo que exige expresar el nivel de confianza:

$$P\tilde{\theta} \sim N(Q, \Omega) \tag{13}$$

El vector de incertidumbre $\tilde{\theta}$ se comporta como una normal con valor esperado $q$ y matriz de covarianza $\Omega$, una matriz diagonal que incluye la varianza de cada opinión. Ello nos permite incluir el nivel de confianza en las opiniones subjetivas. 

El modelo de Black-Litterman nos permite integrar, a través de la estadística bayesiana, dos funciones de distribución, una vinculada la información histórica y otra relacionada con la opiniones subjetivas. Así, la ley de Bayes propone:

$$P(A|B)=\frac{P(B|A)P(A)}{P(B)}\tag{14}$$

donde $P(A|B)$ es la probabilidad de A condicionada a que ocurra B, $P(B|A)$ es la probabiliad de B condicionada a que ocurra A, $P(A)$ y $P(B)$ son las probabilidades no condicionadas. En términos generales $P(A)$ se denomina distribución a priori y $P(B|A)$ es la probabilidad condicional de los datos observados a priori. 

Así, consideramos que la probabilidad a priori es definida como la rentabilidad en equilibrio $P(A)$ y las opiniones como la probabilidad condicional a priori $P(B|A)$. $P(B)$ es una constante de normalización. Por último, $P(A|B)$ es la distribución posterior que buscamos.



